In [1]:
import wandb, os

## Datasets Library
> We will prepare the data for the transformers library

In [2]:
from datasets import load_dataset, Features, ClassLabel, Value, load_metric

In [3]:
wandb.init(project="aws_demo", job_type="get_data")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: capecape (use `wandb login --relogin` to force relogin)


we can grab the preprocessed dataset direcly from wandb:

In [4]:
dataset_path  = wandb.use_artifact("capecape/aws_demo/splitted_dataset:latest").download()

In [5]:
dataset_path

'./artifacts/splitted_dataset:v0'

In [7]:
labels = ["negative", "positive"]
stock_features = Features({'Text': Value('string'), 
                           'labels': ClassLabel(names=labels)})

In [8]:
dataset = load_dataset('csv', data_files={"train": os.path.join(dataset_path,"train.csv"), 
                                          "test": os.path.join(dataset_path, "test.csv")}, 
                       delimiter=',', 
                       features=stock_features)

Using custom data configuration default-d8dfb5a2e7af0ec3
Reusing dataset csv (/home/paperspace/.cache/huggingface/datasets/csv/default-d8dfb5a2e7af0ec3/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

we get a `DatasetDict` object containing our split

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'labels'],
        num_rows: 5212
    })
    test: Dataset({
        features: ['Text', 'labels'],
        num_rows: 579
    })
})

# A Simple bert

In [10]:
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length')

def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/paperspace/.cache/huggingface/datasets/csv/default-d8dfb5a2e7af0ec3/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-055f664b758555b4.arrow
Loading cached processed dataset at /home/paperspace/.cache/huggingface/datasets/csv/default-d8dfb5a2e7af0ec3/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-946972fb275e0871.arrow


In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5212
    })
    test: Dataset({
        features: ['Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 579
    })
})

In [13]:
default_training_args = {
    'per_device_train_batch_size': 64,
    'per_device_eval_batch_size': 64,
    'num_train_epochs': 2,
    'learning_rate': 2e-5,
    'evaluation_strategy': 'epoch',
    'save_strategy': 'epoch',
    'save_total_limit': 2,
    'logging_strategy': 'steps',
    'logging_first_step': True,
    'logging_steps': 5,
    'report_to': 'wandb',
    'fp16':True,
    'dataloader_num_workers':6
}

In [14]:
recall_metric = load_metric("recall")
f1_metric = load_metric('f1')
accuracy_metric = load_metric('accuracy')
precision_metric = load_metric('precision')

In [15]:
from wandb.sdk.integration_utils.data_logging import ValidationDataLogger

validation_inputs = tokenized_datasets['test'].remove_columns(['labels', 'attention_mask', 'input_ids', 'token_type_ids'])
validation_targets = [tokenized_datasets['test'].features['labels'].int2str(x) for x in dataset['test']['labels']]

validation_logger = ValidationDataLogger(inputs = validation_inputs[:],targets = validation_targets)

In [16]:
def compute_metrics(eval_pred):
    "Get a bunch of metrics and log predictions to wandb"
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    recall = recall_metric.compute(predictions=predictions, references=labels, average='macro')['recall']
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro')['f1']
    accuracy = accuracy_metric.compute(predictions=predictions,references=labels)['accuracy']
    precision = precision_metric.compute(predictions=predictions,references=labels,average='macro')['precision']
    
    # convert predictions from class (0, 1) to label (Negative, Positive)
    prediction_labels = [tokenized_datasets['test'].features['labels'].int2str(x.item()) for x in predictions]
    
    # log predictions
    validation_logger.log_predictions(prediction_labels)

    return {
        'recall': recall,
        'f1': f1,
        'accuracy': accuracy,
        'precision': precision
    }

In [17]:
def get_trainer(model, output_dir, tokenizer, data_collator, training_args, train, test):
    "Prepare the hf Trainer"
    training_args = TrainingArguments(
        output_dir=output_dir,
        **training_args
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=train,
        eval_dataset=test,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    return trainer

Let's log predictions at the end of each epoch

In [18]:
def train(train_args=default_training_args, model_name="bert-base-cased"):
    
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    trainer = get_trainer(
        output_dir=f'training_dir',
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        training_args=train_args,
        train=tokenized_datasets['train'],
        test=tokenized_datasets["test"])
    
    trainer.train()

In [19]:
train()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Epoch,Training Loss,Validation Loss,Recall,F1,Accuracy,Precision
1,0.597700,0.587250,0.666808,0.666051,0.683938,0.665405
2,0.461700,0.488913,0.741067,0.745429,0.765112,0.752386


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Text.
***** Running Evaluation *****
  Num examples = 579
  Batch size = 64
Saving model checkpoint to training_dir/checkpoint-82
Configuration saved in training_dir/checkpoint-82/config.json
Model weights saved in training_dir/checkpoint-82/pytorch_model.bin
tokenizer config file saved in training_dir/checkpoint-82/tokenizer_config.json
Special tokens file saved in training_dir/checkpoint-82/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Text.
***** Running Evaluation *****
  Num examples = 579
  Batch size = 64
Saving model checkpoint to training_dir/checkpoint-164
Configuration saved in training_dir/checkpoint-164/config.json
Model weights saved in training_dir/checkpoint-164/pytorch_model.bin
tokenizer config file s

In [20]:
wandb.finish()

eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,▁█
eval/recall,▁█
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
train/learning_rate,███▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁


## Sweeps

In [21]:
import math

In [22]:
sweep_config = {
    'method': 'bayes'
    }

For `bayes`ian Sweeps,
you also need to tell us a bit about your `metric`.
We need to know its `name`, so we can find it in the model outputs
and we need to know whether your `goal` is to `minimize` it
(e.g. if it's the squared error)
or to `maximize` it
(e.g. if it's the accuracy).

In [23]:
metric = {
    'name': 'eval/loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

Once you've picked a `method` to try out new values of the hyperparameters,
you need to define what those `parameters` are.

Most of the time, this step is straightforward:
you just give the `parameter` a name
and specify a list of legal `values`
of the parameter.

For example, when we choose the `optimizer` for our network,
there's only a finite number of options.
Here we stick with the two most popular choices, `adam` and `sgd`.
Even for hyperparameters that have potentially infinite options,
it usually only makes sense to try out
a few select `values`,
as we do here with the hidden `layer_size` and `dropout`.

In [24]:
parameters_dict = {
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.001
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(16),
        'max': math.log(64),
      },
    'epochs': {
        "values": [4,6,8,10]
    }
}
sweep_config['parameters'] = parameters_dict

In [25]:
sweep_id = wandb.sweep(sweep_config, project="aws_demo")

Create sweep with ID: 60gv5p2v
Sweep URL: https://wandb.ai/capecape/aws_demo/sweeps/60gv5p2v


In [26]:
sweep_config

{'method': 'bayes',
 'metric': {'name': 'eval/loss', 'goal': 'minimize'},
 'parameters': {'learning_rate': {'distribution': 'uniform',
   'min': 0,
   'max': 0.001},
  'batch_size': {'distribution': 'q_log_uniform',
   'q': 1,
   'min': 2.772588722239781,
   'max': 4.1588830833596715},
  'epochs': {'values': [4, 6, 8, 10]}}}

In [27]:
def train_sweep(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        
        default_training_args["learning_rate"] = config.learning_rate
        default_training_args['per_device_train_batch_size'] = config.batch_size
        default_training_args['per_device_eval_batch_size'] = config.batch_size
        default_training_args["num_train_epochs"] = config.epochs
        
        train(default_training_args)

In [28]:
wandb.agent(sweep_id, train_sweep, count=5)

wandb: Agent Starting Run: 9zv9oary with config:
wandb: 	batch_size: 42
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0005886648905335285
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/paperspace/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cas

Epoch,Training Loss,Validation Loss,Recall,F1,Accuracy,Precision
1,0.652900,0.682121,0.500000,0.382729,0.620035,0.310017
2,0.689600,0.712495,0.500000,0.275344,0.379965,0.189983
3,0.689700,0.669273,0.500000,0.382729,0.620035,0.310017
4,0.730300,0.665162,0.500000,0.382729,0.620035,0.310017
5,0.683000,0.667391,0.500000,0.382729,0.620035,0.310017
6,0.666900,0.664898,0.500000,0.382729,0.620035,0.310017
7,0.715600,0.664385,0.500000,0.382729,0.620035,0.310017
8,0.693100,0.670746,0.500000,0.382729,0.620035,0.310017
9,0.654000,0.666067,0.500000,0.382729,0.620035,0.310017
10,0.674800,0.664293,0.500000,0.382729,0.620035,0.310017


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Text.
***** Running Evaluation *****
  Num examples = 579
  Batch size = 42
Saving model checkpoint to training_dir/checkpoint-125
Configuration saved in training_dir/checkpoint-125/config.json
Model weights saved in training_dir/checkpoint-125/pytorch_model.bin
tokenizer config file saved in training_dir/checkpoint-125/tokenizer_config.json
Special tokens file saved in training_dir/checkpoint-125/special_tokens_map.json
Deleting older checkpoint [training_dir/checkpoint-82] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Text.
***** Running Evaluation *****
  Num examples = 579
  Batch size = 42
Saving model checkpoint to training_dir/checkpoint-250
Configuration saved in training_dir/checkpoint-250/config.json
Mod

eval/accuracy,█▁████████
eval/f1,█▁████████
eval/loss,▄█▂▁▁▁▁▂▁▁
eval/precision,█▁████████
eval/recall,▁▁▁▁▁▁▁▁▁▁
eval/runtime,▁▁▁▆▁█▁▂▁▁
eval/samples_per_second,█▇█▃█▁█▇▇▇
eval/steps_per_second,█▇█▃█▁█▇▇▇
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁


...

wandb: Agent Starting Run: unvkfmuk with config:
wandb: 	batch_size: 18
wandb: 	epochs: 6
wandb: 	learning_rate: 0.0007055478708026065
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/paperspace/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cas

Epoch,Training Loss,Validation Loss,Recall,F1,Accuracy,Precision
1,0.679700,0.720638,0.500000,0.382729,0.620035,0.310017
2,0.702900,0.743546,0.500000,0.275344,0.379965,0.189983
3,0.656500,0.670645,0.500000,0.382729,0.620035,0.310017
4,0.693500,0.690597,0.500000,0.382729,0.620035,0.310017
5,0.666500,0.665170,0.500000,0.382729,0.620035,0.310017
6,0.702500,0.664235,0.500000,0.382729,0.620035,0.310017


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Text.
***** Running Evaluation *****
  Num examples = 579
  Batch size = 18
Saving model checkpoint to training_dir/checkpoint-290
Configuration saved in training_dir/checkpoint-290/config.json
Model weights saved in training_dir/checkpoint-290/pytorch_model.bin
tokenizer config file saved in training_dir/checkpoint-290/tokenizer_config.json
Special tokens file saved in training_dir/checkpoint-290/special_tokens_map.json
Deleting older checkpoint [training_dir/checkpoint-276] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Text.
***** Running Evaluation *****
  Num examples = 579
  Batch size = 18
Saving model checkpoint to training_dir/checkpoint-580
Configuration saved in training_dir/checkpoint-580/config.json
Mo

eval/accuracy,█▁████
eval/f1,█▁████
eval/loss,▆█▂▃▁▁
eval/precision,█▁████
eval/recall,▁▁▁▁▁▁
eval/runtime,▁▃▁█▆▂
eval/samples_per_second,█▆█▁▃▆
eval/steps_per_second,█▆█▁▃▆
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
